In [13]:
import requests
import pandas as pd
import json
from datetime import datetime

# Carrega o arquivo JSON
list_city = r"C:\Users\sensix\Desktop\PESSOAL\PESSOAL\PORTIFOLIO DATA SCIENCE\pd\pd\Dados\city_coordinates.json"
with open(list_city, 'r') as file:
    city_coordinates = json.load(file)

# Token da API OpenWeatherMap
token = '11f2871a588e33e781d6365014dba754'

# Lista para armazenar os dados de cada cidade
data_list = []

# Inicia o loop sobre as chaves do dicionário city_coordinates
for city, coordinates in city_coordinates.items():
    lat = coordinates["latitude"]
    lon = coordinates["longitude"]
    
    # Faz a solicitação para a API do OpenWeatherMap
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={token}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        aqi = data["list"][0]["main"]
        components = data["list"][0]["components"]
        dt = datetime.fromtimestamp(data["list"][0]["dt"])
        
        # Adiciona os dados à lista
        data_list.append({
            "City": city,
            "Latitude": lat,
            "Longitude": lon,
            "Timestamp": dt,
            "AQI": aqi.get("aqi",None),
            "CO": components.get("co", None),
            "NO": components.get("no", None),
            "NO2": components.get("no2", None),
            "O3": components.get("o3", None),
            "SO2": components.get("so2", None),
            "PM2.5": components.get("pm2_5", None),
            "PM10": components.get("pm10", None),
            "NH3": components.get("nh3", None)

        })
    else:
        print(f"Falha na solicitação para {city}")

# Cria o DataFrame
df_final = pd.DataFrame(data_list)

In [54]:
df_final


,City,Latitude,Longitude,Timestamp,AQI,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
0,"Sao Paulo, Brazil",-23.682795,-46.716998,2024-03-05 19:21:07,4,1455.31,4.30,100.08,4.16,30.28,53.82,92.97,15.07
1,"Rio de Janeiro, Brazil",-22.911014,-43.209373,2024-03-05 19:21:07,3,988.01,0.00,59.63,90.12,26.70,37.70,67.95,11.40
2,"Uberaba, Brazil",-19.753883,-47.939313,2024-03-05 19:21:08,1,310.42,0.01,3.38,24.32,0.18,1.28,1.67,1.54
3,"Paracatu, Brazil",-17.225689,-46.874630,2024-03-05 19:21:08,1,283.72,0.01,3.13,10.55,0.22,1.01,1.15,0.93
4,"Montes Claros, Brazil",-16.737773,-43.859765,2024-03-05 19:21:09,1,216.96,0.01,2.27,8.58,0.30,0.50,1.10,1.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"Diadema, Brazil",-23.686643,-46.624693,2024-03-05 19:21:46,4,1455.31,4.30,100.08,4.16,30.28,53.82,92.97,15.07
77,"Carapicuiba, Brazil",-23.523467,-46.840681,2024-03-05 19:21:47,2,480.65,0.00,8.23,77.25,2.21,19.38,31.29,1.35
78,"Maringa, Brazil",-23.425269,-51.938208,2024-03-05 19:21:48,1,397.21,0.02,5.01,56.51,1.06,9.17,9.97,2.85
79,"Caucaia, Brazil",-3.730056,-38.659308,2024-03-05 19:21:48,1,283.72,0.00,3.51,44.35,0.66,4.14,9.56,0.15


In [68]:
import folium

# Criar um mapa centrado no Brasil
mapa = folium.Map(location=[-14.235, -51.9253], zoom_start=4)

# Converter os dados de AQI para uma lista de tuplas (latitude, longitude, AQI)
dados_aqi = df_final[['Latitude', 'Longitude', 'AQI']].dropna()
dados_aqi = dados_aqi.values.tolist()

# Adicionar o mapa de calor ao mapa
folium.plugins.HeatMap(dados_aqi, gradient={.2: '#2b8318', .4: '#abdda4', .6: '#ffffbf', .8: '#fdae61', .1: '#d7191c'},radius = 20,blur=15).add_to(mapa)

texto_legenda = '''
                <div style="position: fixed; bottom: 50px; left: 50px; background-color: rgba(255, 255, 255, 0.5);
                border-radius: 5px; padding: 10px; z-index: 1000; font-size: 14px;">
                <p><strong>Legenda:</strong></p>
                <p><span style="color: #2b8318;">Bom: 1</span> </p>
                <p><span style="color: #abdda4;">Razóavel: 2</span> </p>
                <p><span style="color: #ffffbf;">Moderado: 3</span> </p>
                <p><span style="color: #fdae61;">Ruim: 4</span> </p>
                <p><span style="color: #d7191c;">Muito Ruim: 5</span> </p>
                </div>
                '''

mapa.get_root().html.add_child(folium.Element(texto_legenda))

# Exibir o mapa
mapa


In [49]:
dados_aqi

[[-23.682794700000002, -46.716998204286014, 4.0],
 [-22.9110137, -43.2093727, 3.0],
 [-19.753883, -47.939313, 1.0],
 [-17.225689, -46.87463, 1.0],
 [-16.737773, -43.859765, 1.0],
 [-15.56246, -54.29859, 1.0],
 [-8.7645, -63.8568, 2.0],
 [0.02552, -51.07463, 1.0],
 [-29.694418, -53.818957, 1.0],
 [-27.105412, -52.620794, 1.0],
 [-24.958088, -53.461454, 1.0],
 [2.81891, -60.68871, 1.0],
 [-9.97377, -67.80932, 1.0],
 [-11.86046, -55.510188, 1.0],
 [-12.2578934, -38.9598047, 1.0],
 [-11.30532, -41.86531, 1.0],
 [-15.792718, -47.88901, 1.0],
 [-27.59092, -48.62056, 1.0],
 [-5.37065, -49.13851, 1.0],
 [-32.039477, -52.104281, 1.0],
 [-2.443112, -54.718477, 1.0],
 [-7.0316, -55.42594, 1.0],
 [-28.259723, -52.415498, 1.0],
 [-31.7465, -52.33766, 1.0],
 [-26.91514, -49.074312, 1.0],
 [-14.863315, -40.848603, 1.0],
 [-17.892597, -51.729764, 1.0],
 [-15.89199, -52.26283, 1.0],
 [-22.236162, -54.823224, 1.0],
 [-13.074924, -55.914069, 1.0],
 [-16.44469, -39.10141, 1.0],
 [-15.3205, -49.14213, 1.0]